# Generate Categories Data

Generate INSERT SQL statements cho bảng categories và lưu vào file `category.insert.sql`

---

## 1. Import Libraries

## 2. Helper Functions

In [1]:
import os
import json
import random
import re
import unicodedata
import psycopg2
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")

print("✅ Đã import tất cả thư viện")

✅ Đã import tất cả thư viện


In [2]:
def slugify(text):
    """Tạo slug từ text tiếng Việt"""
    # Normalize unicode và loại bỏ dấu
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Lowercase và replace spaces với dấu gạch ngang
    text = text.lower()
    text = re.sub(r'[^\w\s-]', '', text)
    text = re.sub(r'[-\s]+', '-', text)
    text = text.strip('-')
    
    return text

def get_next_category_ids(conn, cursor, count=3):
    """Lấy các ID tiếp theo cho categories mới"""
    if not conn:
        return None
    
    try:
        # Lấy giá trị hiện tại của sequence
        cursor.execute("SELECT MAX(id) FROM categories;")
        max_id = cursor.fetchone()[0]
        
        if max_id is None:
            next_id = 1
        else:
            next_id = max_id + 1
        
        ids = list(range(next_id, next_id + count))
        print(f"📊 ID hiện tại lớn nhất: {max_id}")
        print(f"🆔 {count} ID tiếp theo sẽ là: {ids}")
        return ids
    except Exception as e:
        print(f"❌ Lỗi khi lấy sequence: {e}")
        return None

print("✅ Đã định nghĩa các hàm hỗ trợ: slugify(), get_next_category_ids()")

✅ Đã định nghĩa các hàm hỗ trợ: slugify(), get_next_category_ids()


## 3. Main Execution

In [3]:
# ============================================
# BƯỚC 1: Kết nối Supabase
# ============================================
if SUPABASE_CONNECTION_STRING:
    try:
        conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
        cursor = conn.cursor()
        print("✅ Kết nối Supabase thành công!")
    except Exception as e:
        print(f"❌ Lỗi kết nối Supabase: {e}")
        conn = None
        cursor = None
else:
    print("⚠️ Không tìm thấy SUPABASE_CONNECTION_STRING trong .env")
    conn = None
    cursor = None

# ============================================
# BƯỚC 2: Lấy IDs tiếp theo và load images
# ============================================
new_parent_ids = get_next_category_ids(conn, cursor, 3)

try:
    product_json_path = os.path.join('..', 'product', 'product.api.json')
    with open(product_json_path, 'r', encoding='utf-8') as f:
        product_data = json.load(f)
    
    image_urls = [p.get('image_url') for p in product_data.get('data', []) if p.get('image_url')]
    print(f"📷 Đã load {len(image_urls)} images từ product.api.json")
except Exception as e:
    print(f"❌ Lỗi load images: {e}")
    image_urls = []

# ============================================
# BƯỚC 3: Định nghĩa categories
# ============================================
if new_parent_ids and len(new_parent_ids) >= 3:
    id_3, id_4, id_5 = new_parent_ids
    
    parent_categories = [
        {"id": 1, "name": "Điện Tử", "existing": True},
        {"id": 2, "name": "Thời Trang", "existing": True},
        {"id": id_3, "name": "Nội Thất & Gia Dụng", "existing": False},
        {"id": id_4, "name": "Thể Thao & Du Lịch", "existing": False},
        {"id": id_5, "name": "Sách & Văn Phòng Phẩm", "existing": False},
    ]
    
    child_categories = {
        1: ["Điện Thoại", "Laptop", "Máy Tính Bảng", "Phụ Kiện Điện Tử", "Thiết Bị Âm Thanh"],
        2: ["Áo Nam", "Áo Nữ", "Quần Jeans", "Giày Dép", "Phụ Kiện Thời Trang"],
        id_3: ["Bàn Ghế", "Sofa", "Đồ Dùng Nhà Bếp", "Đèn Trang Trí", "Chăn Ga Gối"],
        id_4: ["Dụng Cụ Tập Gym", "Xe Đạp", "Giày Thể Thao", "Phụ Kiện Du Lịch", "Dụng Cụ Camping"],
        id_5: ["Sách Văn Học", "Sách Kỹ Năng", "Văn Phòng Phẩm", "Dụng Cụ Học Tập", "Sổ Tay"]
    }
    
    print("✅ Đã định nghĩa categories")
    print(f"📦 {len(parent_categories)} parent categories")
    print(f"📦 {sum(len(v) for v in child_categories.values())} child categories")
else:
    print("⚠️ Không thể lấy IDs mới")
    parent_categories = []
    child_categories = {}

# ============================================
# BƯỚC 4: Generate SQL INSERT statements
# ============================================
try:
    sql_statements = []
    sql_statements.append("-- Auto-generated INSERT statements for categories table")
    sql_statements.append("-- Generated at: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    sql_statements.append("")
    sql_statements.append("-- ============================================")
    sql_statements.append("-- PARENT CATEGORIES (3 categories mới)")
    sql_statements.append("-- ============================================")
    
    for parent in parent_categories:
        if not parent.get('existing', False):
            name = parent['name']
            slug = slugify(name)
            cat_image = random.choice(image_urls) if image_urls else ''
            
            insert_sql = f"""
INSERT INTO categories (name, parent_id, status, deleted, description, slug, cat_image)
VALUES ('{name}', NULL, 'active', FALSE, 'Danh mục {name}', '{slug}', '{cat_image}');
"""
            sql_statements.append(insert_sql.strip())
    
    sql_statements.append("")
    sql_statements.append("-- ============================================")
    sql_statements.append("-- CHILD CATEGORIES (25 categories con)")
    sql_statements.append("-- ============================================")
    
    for parent in parent_categories:
        parent_id = parent['id']
        parent_name = parent['name']
        
        sql_statements.append(f"\n-- Child categories của '{parent_name}' (parent_id = {parent_id})")
        
        children = child_categories.get(parent_id, [])
        for child_name in children:
            slug = slugify(child_name)
            cat_image = random.choice(image_urls) if image_urls else ''
            
            insert_sql = f"""
INSERT INTO categories (name, parent_id, status, deleted, description, slug, cat_image)
VALUES ('{child_name}', {parent_id}, 'active', FALSE, '{child_name} thuộc {parent_name}', '{slug}', '{cat_image}');
"""
            sql_statements.append(insert_sql.strip())
    
    with open('category.insert.sql', 'w', encoding='utf-8') as f:
        f.write('\n\n'.join(sql_statements))
    
    parent_count = sum(1 for p in parent_categories if not p.get('existing', False))
    child_count = sum(len(v) for v in child_categories.values())
    
    print(f"\n✅ Đã tạo file category.insert.sql")
    print(f"📊 {parent_count} parent categories mới")
    print(f"📊 {child_count} child categories")
    print(f"📊 Tổng cộng: {parent_count + child_count} câu lệnh INSERT")
    
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

✅ Kết nối Supabase thành công!
📊 ID hiện tại lớn nhất: 6
🆔 3 ID tiếp theo sẽ là: [7, 8, 9]
📷 Đã load 100 images từ product.api.json
✅ Đã định nghĩa categories
📦 5 parent categories
📦 25 child categories

✅ Đã tạo file category.insert.sql
📊 3 parent categories mới
📊 25 child categories
📊 Tổng cộng: 28 câu lệnh INSERT


In [10]:
# Generate SQL chỉ cho child categories của 7, 8, 9
try:
    # Set parent IDs thủ công (đã biết là 7, 8, 9)
    id_7, id_8, id_9 = 7, 8, 9
    
    # Chỉ lấy categories 7, 8, 9
    parent_categories_new = [
        {"id": id_7, "name": "Nội Thất & Gia Dụng"},
        {"id": id_8, "name": "Thể Thao & Du Lịch"},
        {"id": id_9, "name": "Sách & Văn Phòng Phẩm"},
    ]
    
    child_categories_new = {
        id_7: ["Bàn Ghế", "Sofa", "Đồ Dùng Nhà Bếp", "Đèn Trang Trí", "Chăn Ga Gối"],
        id_8: ["Dụng Cụ Tập Gym", "Xe Đạp", "Giày Thể Thao", "Phụ Kiện Du Lịch", "Dụng Cụ Camping"],
        id_9: ["Sách Văn Học", "Sách Kỹ Năng", "Văn Phòng Phẩm", "Dụng Cụ Học Tập", "Sổ Tay"]
    }
    
    # Load images
    try:
        product_json_path = os.path.join('..', 'product', 'product.api.json')
        with open(product_json_path, 'r', encoding='utf-8') as f:
            product_data = json.load(f)
        image_urls = [p.get('image_url') for p in product_data.get('data', []) if p.get('image_url')]
    except:
        image_urls = []
    
    sql_statements = []
    sql_statements.append("-- Auto-generated INSERT statements for child categories of 7, 8, 9")
    sql_statements.append("-- Generated at: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    sql_statements.append("")
    
    # Chỉ insert child categories
    for parent in parent_categories_new:
        parent_id = parent['id']
        parent_name = parent['name']
        
        sql_statements.append(f"\n-- Child categories của '{parent_name}' (parent_id = {parent_id})")
        
        children = child_categories_new.get(parent_id, [])
        for child_name in children:
            slug = slugify(child_name)
            cat_image = random.choice(image_urls) if image_urls else ''
            
            insert_sql = f"""
INSERT INTO categories (name, parent_id, status, deleted, description, slug, cat_image)
VALUES ('{child_name}', {parent_id}, 'active', FALSE, '{child_name} thuộc {parent_name}', '{slug}', '{cat_image}');
"""
            sql_statements.append(insert_sql.strip())
    
    # Ghi vào file mới
    with open('category_child_789.insert.sql', 'w', encoding='utf-8') as f:
        f.write('\n\n'.join(sql_statements))
    
    child_count = sum(len(v) for v in child_categories_new.values())
    print(f"✅ Đã tạo file category_child_789.insert.sql")
    print(f"📊 {child_count} child categories cho parent 7, 8, 9")
    
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

✅ Đã tạo file category_child_789.insert.sql
📊 15 child categories cho parent 7, 8, 9


## 3.5. Generate SQL cho child categories của 7, 8, 9 thôi

In [14]:
# Đọc và thực thi các câu lệnh INSERT từ category_child_789.insert.sql
try:
    # Import thư viện cần thiết
    import os
    import psycopg2
    from dotenv import load_dotenv
    
    # Load environment variables
    load_dotenv()
    SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")
    
    # Luôn tạo connection mới
    print("⚙️ Đang tạo kết nối Supabase...")
    if SUPABASE_CONNECTION_STRING:
        conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
        cursor = conn.cursor()
        print("✅ Kết nối Supabase thành công!")
    else:
        print("❌ Không tìm thấy SUPABASE_CONNECTION_STRING trong .env")
        raise Exception("Missing SUPABASE_CONNECTION_STRING")
    
    # Đọc file SQL
    with open('category_child_789.insert.sql', 'r', encoding='utf-8') as f:
        sql_content = f.read()
    
    # Tách các câu lệnh INSERT (bỏ qua comments và dòng trống)
    sql_lines = sql_content.split('\n')
    sql_statements = []
    current_statement = []
    
    for line in sql_lines:
        line = line.strip()
        # Bỏ qua comment và dòng trống
        if line.startswith('--') or not line:
            continue
        
        current_statement.append(line)
        
        # Nếu gặp dấu ';' thì kết thúc statement
        if line.endswith(';'):
            full_statement = ' '.join(current_statement)
            sql_statements.append(full_statement)
            current_statement = []
    
    print(f"📄 Đã đọc {len(sql_statements)} câu lệnh INSERT từ category_child_789.insert.sql")
    
    # Thực thi từng câu lệnh
    success_count = 0
    error_count = 0
    
    for idx, statement in enumerate(sql_statements, 1):
        try:
            cursor.execute(statement)
            conn.commit()
            success_count += 1
            print(f"✅ [{idx}/{len(sql_statements)}] Thực thi thành công")
        except Exception as e:
            conn.rollback()
            error_count += 1
            print(f"❌ [{idx}/{len(sql_statements)}] Lỗi: {e}")
    
    print(f"\n📊 Kết quả:")
    print(f"   ✅ Thành công: {success_count}")
    print(f"   ❌ Thất bại: {error_count}")
    print(f"   📊 Tổng: {len(sql_statements)}")
    
    # Đóng connection
    cursor.close()
    conn.close()
    print("\n🔒 Đã đóng kết nối Supabase")
        
except FileNotFoundError:
    print("❌ Không tìm thấy file category_child_789.insert.sql")
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

⚙️ Đang tạo kết nối Supabase...
✅ Kết nối Supabase thành công!
📄 Đã đọc 15 câu lệnh INSERT từ category_child_789.insert.sql
✅ [1/15] Thực thi thành công
✅ [2/15] Thực thi thành công
✅ [3/15] Thực thi thành công
✅ [4/15] Thực thi thành công
✅ [5/15] Thực thi thành công
✅ [6/15] Thực thi thành công
✅ [7/15] Thực thi thành công
✅ [8/15] Thực thi thành công
✅ [9/15] Thực thi thành công
✅ [10/15] Thực thi thành công
✅ [11/15] Thực thi thành công
✅ [12/15] Thực thi thành công
✅ [13/15] Thực thi thành công
✅ [14/15] Thực thi thành công
✅ [15/15] Thực thi thành công

📊 Kết quả:
   ✅ Thành công: 15
   ❌ Thất bại: 0
   📊 Tổng: 15

🔒 Đã đóng kết nối Supabase


## 4. Execute SQL Statements to Supabase